### Import libraries

In [1]:
! pip install torchmetrics torcheval

In [2]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
from torch import nn
import torch.optim as optim
from torcheval.metrics.functional import multiclass_accuracy, multiclass_auprc
from torchmetrics import JaccardIndex

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Deep_learning_course/Lung_segmentation/classification_segmentation/')
from custom_load import LungDatasetLoader
from unet_mobilenet import NetCovid
from fit_func import FitTrainEvalCovidNet

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 160MB/s]
/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://gi

### Import data

In [5]:
path_to_zip_folder = '/content/drive/MyDrive/Deep_learning_course/Lung_segmentation/data/m3ex02-data.zip'
current_folder_path = '/content/localpath/'
!unzip -q $path_to_zip_folder -d $current_folder_path

### Define parameters

In [34]:
# data parameters
root_path: str = '/content/localpath/m3ex02-data/'
test_path: str = os.path.join(root_path, 'Test/')
train_path: str = os.path.join(root_path, 'Train/')
val_path: str = os.path.join(root_path, 'Val/')

workers: int = 12
batch_size: int = 182
# 182

# model parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
jaccard_idx = JaccardIndex(task="BINARY").to(device)
jaccard_idx.__name__ = "jaccard_index"
metrics: dict = {"classification": [multiclass_accuracy, multiclass_auprc], "segmentation": [jaccard_idx]}
save_path: str = "/content/drive/MyDrive/Deep_learning_course/Lung_segmentation/classification_segmentation/models/"
name_model: str = "model_test_4.pkl"
name_load: str = "model_test_3.pkl"
in_channels: int = 1
classes: int = 3
seg_channels: int = 1
EPOCHS: int = 40

### Load data

In [35]:
data_train = LungDatasetLoader(train_path, batch_size=batch_size, workers=workers, pin_memory_device=device)
data_val = LungDatasetLoader(val_path, batch_size=batch_size, workers=workers, pin_memory_device=device)
data_test = LungDatasetLoader(test_path, batch_size=batch_size, workers=workers, pin_memory_device=device)

In [36]:
len(data_train), len(data_val), len(data_test)

(120, 30, 38)

### Training

In [37]:
model = NetCovid(in_channels=in_channels, num_classes=classes, seg_channels=seg_channels)
model.load_state_dict(torch.load(os.path.join(save_path, name_load))["model"])
model = model.to(device)

Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [42]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-6)

In [43]:
# Initialize optimizer
optimization = FitTrainEvalCovidNet(
    model=model,
    loss_seg=criterion,
    loss_class=criterion,
    weights=[0.3, 0.7],
    optimizer=optimizer,
    metrics=metrics,
    path_save=os.path.join(save_path, name_model),
    device=device
)


In [ ]:
# train-evaluation
trained = optimization.fit_train_eval(
    data_train=data_train,
    data_eval=data_val,
    epochs=EPOCHS
)